## 필요한 모듈 import

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

pd.options.display.max_columns = None

## 실습을 위한 데이터셋 로드

In [ ]:
from pycaret.datasets import get_data
dataset = get_data('insurance')

In [ ]:
# 데이터셋 크기 출력
dataset.shape

In [ ]:
train = dataset.sample(frac=0.8, random_state=123)
test = dataset.drop(train.index)
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

print('학습용 데이터셋: ' + str(train.shape))
print('예측용 데이터셋: ' + str(test.shape))

## 설정: setup()

머신러닝 예측 방식에 따라 다음 중 하나를 골라 import 합니다.

In [ ]:
from pycaret.regression import *        # 회귀

**`setup` 함수**

- `data`: 학습할 데이터셋을 지정합니다.
- `target`: 예측할 대상(target) 컬럼을 지정합니다.
- `session_id`: SEED 값을 지정합니다.

In [ ]:
train.head(3)

In [ ]:
clf = setup(data=train, 
            target='charges', 
            session_id=123, 
            verbose=False,
            ) 

## 모든 모델에 대한 학습 compare_models()

`compare_models`
- `sort`: 정렬 기준이 되는 평가지표를 설정합니다.
- `n_select`: 상위 N개의 알고리즘을 선택합니다.
- `fold`: Cross Validation 평가 Fold의 개수를 지정합니다.
- `round`: 결과를 소수 N째자리 까지 반올림하여 표기합니다.

In [ ]:
best_models = compare_models(n_select=3, fold=5, round=2, sort='mae')

### 모델 블렌딩 blend_models

- `compare_models`로 추출된 best 모델에 대하여 모델 블렌딩하여 성능 개선
- `Soft` voting 방식으로 `estimator_list`에 적용된 모델을 앙상블
- `Voting Ensemble`

In [ ]:
blended_models = blend_models(best_models, fold=5)

## 단일 모델 생성 create_models()

단일 모델을 생성하기 위해서는 `create_model`로 생성할 수 있습니다.

In [ ]:
models()

In [ ]:
dt = create_model('dt')

## 모델 분석: interpret_model()

### 특성 중요도 Feature Importances

각 특성별 종속변수(Y)에 미치는 영향도를 계산

In [ ]:
plot_model(dt, plot='feature')

In [ ]:
# 그림을 png로 저장하기 위해서는 save=True 입력
plot_model(dt, plot='feature', save=True)

### 예측 값과 실제 값을 비교

In [ ]:
plot_model(dt, plot='error')

### Shap Value 분석

SHAP (SHapley Additive exPlanations) 값은 머신 러닝 모델의 예측을 개별적으로 설명하는 방법 중 하나입니다. 이 값은 샘플의 각 특성이 예측에 어떤 영향을 미쳤는지를 설명하는 중요한 정보를 제공합니다.

SHAP 값은 Shapley 값이라는 개념에 기반합니다. Shapley 값은 게임 이론에서 유래한 개념으로, 각각의 기여도가 공정하게 분배되는 것을 보장하는 적합한 방법을 제공합니다. Shapley 값은 각 특성의 기여도를 측정하는 데 사용됩니다.

In [ ]:
interpret_model(dt)

In [ ]:
# 상관관계 분석
interpret_model(dt, plot = 'correlation')

In [ ]:
# feature 에 분석을 원하는 컬럼명을 대입합니다
interpret_model(dt, plot = 'correlation', feature = 'bmi')

### PDP 란?

PDP는 대상 특성의 값이 모델의 예측값에 미치는 영향력을 시각화합니다. 일반적으로 PDP는 대상 특성의 범위를 나열하여 특성의 범위에 따른 모델의 출력값의 변화를 보여줍니다. PDP를 그림으로 나타내면 x축은 대상 특성의 값이 변화하는 범위를 나타내고, y축은 모델의 출력값을 나타냅니다. PDP를 통해 특정 특성이 모델의 예측값에 미치는 영향력을 파악할 수 있습니다.

In [ ]:
interpret_model(dt, plot = 'pdp', feature='bmi')

In [ ]:
interpret_model(dt, plot = 'pdp', feature='age')

## 모델 예측: predict_model()

- `Label`에 예측된 결과를 확인할 수 있습니다.
- `Score`에 예측된 결과의 **확률** 값을 확인할 수 있습니다.

In [ ]:
prediction = predict_model(data=test, estimator=blended_models)